In [19]:
import numpy as np
from scipy.signal import stft
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

data = np.load(r"D:\KAUST\semster2\ML_IN_GEO\SiesmicEventsClassification_Normalized.npz")
X = data['data']  
y = data['label']  

# STFT parameters
fs = 1.0          # sampling frequency 
nperseg = 64      # window length
noverlap = 32     # overlap

X_stft_list = []

for i in range(X.shape[0]):
    f, t, Zxx = stft(X[i], fs=fs, nperseg=nperseg, noverlap=noverlap)
    # Zxx shape: (freq_bins, time_frames), complex
    X_stft_list.append(Zxx)

X_stft = np.stack(X_stft_list, axis=0)  # (N, F, T_frames), complex

# Split into real and imaginary parts (like your FFT version)
X_stft_real = X_stft.real
X_stft_imag = X_stft.imag

# Option A: concatenate along frequency axis
X_stft_combined = np.concatenate([X_stft_real, X_stft_imag], axis=1)
# Shape: (N, 2*F, T_frames)

N, F2, T = X_stft_combined.shape
X_stft_flat = X_stft_combined.reshape(N, F2*T)  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
   X_stft_flat, y, test_size=0.2, random_state=42, shuffle=True
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class SeismicNet(nn.Module):
    def __init__(self, input_size, hidden_layers=[64, 32], output_size=2):
        super().__init__()
        layers = []
        last_size = input_size
        for h in hidden_layers:
            layers.append(nn.Linear(last_size, h))
            layers.append(nn.ReLU())
            last_size = h
        layers.append(nn.Linear(last_size, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

#train
def train_model(model, X_train, y_train, epochs=20, batch_size=32, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epochs):
        permutation = torch.randperm(X_train.size()[0])
        correct = 0
        total = 0
        for i in range(0, X_train.size()[0], batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
          
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
        
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Accuracy: {epoch_acc:.4f}")
        
    return model

#test
def test_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        preds = torch.argmax(outputs, dim=1)
        correct = (preds == y_test).sum().item()
        acc = correct / y_test.size(0)
    return acc

In [14]:
#config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor  = X_test_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)


hidden_layer_configs = [
    # 1 hidden layer
    [16], [32], [64], [128], [256],

    # 2 hidden layer
    [32, 16], [64, 32], [128, 64], [256, 128], [128, 32], [64, 16],

    # 3 hidden layer
    [64, 32, 16], [128, 64, 32], [256, 128, 64], [128, 64, 16], [64, 32, 16],

    # 4 hidden layer
    [128, 64, 32, 16], [256, 128, 64, 32], [64, 64, 32, 16],
]

batch_sizes = [16, 32, 64]
learning_rates = [0.001, 0.005, 0.01]

best_acc = 0
best_config = None
best_batch = None
best_lr = None

input_size = X_train_tensor.shape[1]


for config in hidden_layer_configs:
    for batch_size in batch_sizes:
        for lr in learning_rates:
            print(f"\nTraining with hidden layers: {config}, batch size: {batch_size}, lr: {lr}")
            
            
            model = SeismicNet(input_size=input_size, hidden_layers=config)
            model = model.to(device)
            
            
            model = train_model(model, X_train_tensor, y_train_tensor,
                                epochs=20, batch_size=batch_size, lr=lr)
            
            
            acc = test_model(model, X_test_tensor, y_test_tensor)
            print(f"Test Accuracy: {acc:.4f}")
            
            
            if acc > best_acc:
                best_acc = acc
                best_config = config
                best_batch = batch_size
                best_lr = lr

print("\n=============================")
print("Best hidden layer config:", best_config)
print("Best batch size:", best_batch)
print("Best learning rate:", best_lr)
print("Best test accuracy:", best_acc)

Using device: cuda

Training with hidden layers: [16], batch size: 16, lr: 0.001
Epoch 1/20, Accuracy: 0.4963
Epoch 2/20, Accuracy: 0.8625
Epoch 3/20, Accuracy: 0.9663
Epoch 4/20, Accuracy: 0.9912
Epoch 5/20, Accuracy: 0.9950
Epoch 6/20, Accuracy: 0.9950
Epoch 7/20, Accuracy: 0.9962
Epoch 8/20, Accuracy: 0.9975
Epoch 9/20, Accuracy: 0.9975
Epoch 10/20, Accuracy: 0.9975
Epoch 11/20, Accuracy: 0.9975
Epoch 12/20, Accuracy: 0.9975
Epoch 13/20, Accuracy: 0.9975
Epoch 14/20, Accuracy: 0.9975
Epoch 15/20, Accuracy: 0.9975
Epoch 16/20, Accuracy: 0.9975
Epoch 17/20, Accuracy: 0.9975
Epoch 18/20, Accuracy: 0.9975
Epoch 19/20, Accuracy: 0.9975
Epoch 20/20, Accuracy: 0.9975
Test Accuracy: 0.4950

Training with hidden layers: [16], batch size: 16, lr: 0.005
Epoch 1/20, Accuracy: 0.4900
Epoch 2/20, Accuracy: 0.9400
Epoch 3/20, Accuracy: 0.9925
Epoch 4/20, Accuracy: 0.9950
Epoch 5/20, Accuracy: 0.9950
Epoch 6/20, Accuracy: 0.9950
Epoch 7/20, Accuracy: 0.9950
Epoch 8/20, Accuracy: 0.9962
Epoch 9/20, 

In [20]:
model.eval()

with torch.no_grad():
    outputs = model(X_test_tensor)          
    preds = torch.argmax(outputs, dim=1)    
preds_cpu = preds.cpu().numpy()
y_test_cpu = y_test_tensor.cpu().numpy()

In [21]:
cm = confusion_matrix(y_test_cpu, preds_cpu)
print("Confusion Matrix:")
print(cm)

precision = precision_score(y_test_cpu, preds_cpu, average='weighted')
recall = recall_score(y_test_cpu, preds_cpu, average='weighted')
f1 = f1_score(y_test_cpu, preds_cpu, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Confusion Matrix:
[[51 51]
 [42 56]]
Precision: 0.5361
Recall:    0.5350
F1 Score:  0.5345
